<a href="https://colab.research.google.com/github/irissihuichen/Retrofit-Kendall/blob/main/RetrofitKendall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries



the ones that are not in Colaborator


In [7]:
!pip install geopandas matplotlib numpy shapely matplotlib folium pandas

In [8]:
!pip install -U ipykernel
!pip install modin[dask]
!pip install -q streamlit
!pip install pydeck

In [ ]:
!python --version

Python 3.7.12


Import and define syntax

In [9]:
from shapely.geometry import Point, Polygon
from gzip import GzipFile
import geopandas as gpd
import pandas as pd
import numpy as np 
import urllib.request as ur 
import pandas as pd 
import folium
import matplotlib.pyplot as plt
from geopandas import GeoDataFrame
import pydeck as pdk


# Visualizing Data

**Creating base map from geojson file**

In [16]:
# Read geojson file 
fp = 'https://raw.githubusercontent.com/irissihuichen/Retrofit-Kendall/main/kendall_wgs1984.geojson'
data = gpd.read_file(fp)

# Read geometry data and convert to coordinates
data['coords'] = data['geometry'].apply(lambda x: x.representative_point().coords[:][0])
data.head()

,OBJECTID_1,TYPE,BASE_ELEV,ELEV_SL,TOP_SL,ELEV_GL,TOP_GL,BldgID,EditDate,created_us,created_da,last_edite,last_edi_1,BldgID2,OBJECTID,report_year,BldgID_1,map_lot,yr_added,report_rcvd,report_type,bldgs_included,parent_bldg_ID,assessor_addr,prop_type,yr_built,prop_gsf,engy_star_score,elec_grid_prch_kWh,elec_grid_prch_kBtu,ng_therms,ng_kBtu,oil1_kBtu,oil2_kBtu,oil4_kBtu,oil56_kBtu,dies2_kBtu,kero_kBtu,dist_chil_wat_kBtu,dist_steam_kBtu,elec_site_renw_kBtu,site_engy_kBtu,norm_weather_site_engy_kBtu,site_EUI_kBtu_psf,norm_weath_site_EUI_kBtu_psf,source_engy_kBtu,norm_weath_source_engy_kBtu_psf,source_EUI_kBtu_psf,norm_weath_source_EUI,tota_ghg_tonnesCO2e,tot_GHG_intensity_kgCO2e_psf,water_use_kgal,water_intensity_gal_psf,BldgID2_1,Shape_Length,Shape_Area,geometry,coords
0,1,BLDG,9.8,53.1,88.0,43.3,78.2,668-50,668-,GISDATA,1587945600000,GISDATA,1587945600000,668-50,1089,2020,668-50,52A-21,2015,Yes,Single,NA,NA,60 Vassar St,Education,1917.0,73703.0,NaN,NaN,NaN,30.999999,3100.0,None,NaN,None,None,NaN,None,NaN,20293223.9,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,849.0,11.52,668-50,642.357186,23895.820952,"POLYGON ((-71.08818 42.35956, -71.08818 42.359...","(-71.0883162466319, 42.35937042717477)"
1,2,BLDG,8.0,44.4,63.5,36.4,55.5,696-7,696-,GISDATA,1587945600000,GISDATA,1587945600000,696-7,680,2020,696-7,52A-19,2015,Yes,Single,NA,NA,111 Memorial Dr,Education,1917.0,18643.0,NaN,157825.0,538498.9,NaN,NaN,None,NaN,None,None,NaN,None,NaN,6600432.3,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.00,696-7,482.230028,7016.927026,"POLYGON ((-71.08679 42.35973, -71.08677 42.359...","(-71.08697341003202, 42.35964325960574)"
2,3,BLDG,10.7,26.8,28.0,16.1,17.3,696-6,696-,GISDATA,1587945600000,GISDATA,1587945600000,696-6,125,2020,696-6,52A-19,2019,No,NA,NA,NA,111 Memorial Dr,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,696-6,73.340090,300.621582,"POLYGON ((-71.08684 42.35981, -71.08688 42.359...","(-71.08687566072912, 42.35982809842288)"
3,4,BLDG,10.7,76.0,80.2,65.3,69.5,696-5,696-,GISDATA,1587945600000,GISDATA,1587945600000,696-5,211,2020,696-5,52A-19,2015,Yes,Single,NA,NA,111 Memorial Dr,Education,1916.0,64671.0,NaN,599347.9,2044975.3,532.999965,53300.0,None,NaN,None,None,NaN,None,NaN,5790900.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,2981.0,46.09,696-5,831.397182,13581.219351,"POLYGON ((-71.08673 42.36012, -71.08677 42.360...","(-71.08727648025628, 42.35993362539505)"
4,5,BLDG,9.1,31.2,30.7,22.1,21.6,668-17,668-,GISDATA,1587945600000,GISDATA,1587945600000,668-17,701,2020,668-17,52A-21,2015,No,Single,NA,NA,60 Vassar St,Education,1939.0,6075.0,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,86.8,14.28,668-17,179.555973,1517.399860,"POLYGON ((-71.09349 42.36019, -71.09345 42.360...","(-71.09339247153862, 42.360129314463585)"


In [17]:
# Select data only for drawing map
kendall_map = data[['BldgID', 'TYPE', 'source_EUI_kBtu_psf', 'Shape_Length', 'Shape_Area', 'geometry','coords']]
kendall_map.head()

,BldgID,TYPE,source_EUI_kBtu_psf,Shape_Length,Shape_Area,geometry,coords
0,668-50,BLDG,0.0,642.357186,23895.820952,"POLYGON ((-71.08818 42.35956, -71.08818 42.359...","(-71.0883162466319, 42.35937042717477)"
1,696-7,BLDG,0.0,482.230028,7016.927026,"POLYGON ((-71.08679 42.35973, -71.08677 42.359...","(-71.08697341003202, 42.35964325960574)"
2,696-6,BLDG,NaN,73.340090,300.621582,"POLYGON ((-71.08684 42.35981, -71.08688 42.359...","(-71.08687566072912, 42.35982809842288)"
3,696-5,BLDG,0.0,831.397182,13581.219351,"POLYGON ((-71.08673 42.36012, -71.08677 42.360...","(-71.08727648025628, 42.35993362539505)"
4,668-17,BLDG,0.0,179.555973,1517.399860,"POLYGON ((-71.09349 42.36019, -71.09345 42.360...","(-71.09339247153862, 42.360129314463585)"


In [ ]:
# Visualize the building footprints
kendall_map.plot(figsize=(10, 10))

**Selecting geospatial index for display**

In [ ]:
# Define dataframe from file
df = GeoDataFrame(kendall_map)
# Read coordinates
df['coords'] = df['geometry'].apply(lambda x: x.representative_point().coords[:][0])

# Draw the map
fig, ax = plt.subplots(figsize=(15, 18))
df.plot(column='source_EUI_kBtu_psf', cmap='OrRd', linewidth=0.8, ax=ax, edgecolor='0.8')
ax.axis('off')

# Create colorbar as a legend
scalarMap = plt.cm.ScalarMappable(cmap='OrRd', norm=plt.Normalize(vmin=0, vmax=df['source_EUI_kBtu_psf'].max()))
plt.colorbar(scalarMap, fraction=0.036, pad=0.04)

# Show the map                            
plt.show()

In [ ]:
# Drop the row with the abnormal max number
df = df.drop(df['source_EUI_kBtu_psf'].idxmax())

# Draw the map again
fig, ax = plt.subplots(figsize=(15, 18))

df.plot(column='source_EUI_kBtu_psf', cmap='OrRd', linewidth=0.8, ax=ax, edgecolor='0.8')
ax.axis('off')

# Create colorbar as a legend
scalarMap = plt.cm.ScalarMappable(cmap='OrRd', norm=plt.Normalize(vmin=0, vmax=df['source_EUI_kBtu_psf'].max()))
plt.colorbar(scalarMap, fraction=0.036, pad=0.04)

# Show the map
plt.show()

# Folium Map

# Streamlit

Sample:
new = 'C:/Users/Elence/kendall_wgs1984.json'
f = open(new)
sample = json.load(f)
sample

In [ ]:
import streamlit as st

st.title('Kendall Retrofitting')
st.subheader('Building Vintage')
pre1980 = st.slider('Pre_1980', 0.0, 1.0, 0.2)
post1980 = st.slider('Post_1980', 0.0, 1.0, 0.2) 
new2004 = st.slider('New2004', 0.0, 1.0, 0.2)

# st.write('Slider number:', pre1980)


# values = st.sidebar.slider('EUI', float(kendall_new.source_EUI_kBtu_psf.min()), 1000., (50., 300.))
# # f = px.histogram(df.query(f”price.between{values}”), x=”price”, nbins=15, title=”Price distribution”)
# # f.update_xaxes(title=”Price”)
# # f.update_yaxes(title=”No. of listings”)
# # st.plotly_chart(f)

In [15]:
fp = 'https://raw.githubusercontent.com/irissihuichen/Retrofit-Kendall/main/kendall_new2.csv'
kendall_new2 = pd.read_csv(fp)

In [ ]:
# Create circlemarker map
m = folium.Map(location = [42.36, -71.085],
               zoom_start = 15)

for La, Lon, value, name in zip(kendall_new2.lat, kendall_new2.lon, 
                                kendall_new2.EUI_base, kendall_new2.type):
    folium.CircleMarker(location=[La, Lon], 
                        radius=value*0.005, 
                        popup =(str(name).capitalize() + str(value)),
                        color = 'crimson',
                        fill_color ='red').add_to(m)
# Generate map
m.save('circular.html')                        
m

In [ ]:
m2 = folium.Map(location = [42.36, -71.085],
               zoom_start = 15)

for _, r in kendall_new2.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
    style_function = lambda x: {'fillColor': 'orange'})

    folium.Popup(r['EUI_base']).add_to(geo_j)
    geo_j.add_to(m2)
m2

In [ ]:
from branca.colormap import linear
nbh_count_colormap = linear.RdYlBu_11.scale(min(kendall_new2['EUI_base']), max(kendall_new2['EUI_base']))

nbh_locs_map = folium.Map(location = [42.36, -71.085], zoom_start = 15)

style_function = lambda x: {
    'fillColor': nbh_count_colormap(x['properties']['EUI_base']),
    'color': 'black',
    'weight': 1.5,
    'fillOpacity': 0.7
}

folium.GeoJson(
    kendall_new2,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['EUI_base'],
        aliases=['EUI'],
        localize=True
    )).add_to(nbh_locs_map)

nbh_count_colormap.add_to(nbh_locs_map)
nbh_count_colormap.caption = 'EUI'
nbh_count_colormap.add_to(nbh_locs_map)
nbh_locs_map

# Radar Chart Fromat Test

In [ ]:
import plotly.express as px
df = pd.DataFrame(dict(
    r=[1, 5, 2, 2, 3, 2, 2, 2, 2],
    theta=['housing -1980','housing 1980-2004','housing 2004-',
           'office -1980','office 1980-2004','office 2004-',
           'laboratory -1980','laboratory 1980-2004','laboratory 2004-']))
fig = px.line_polar(df, r='r', theta='theta', line_close=True)
fig.show()

In [ ]:
# Multiple trace radar chart
import plotly.graph_objects as go

categories = ['housing -1980','housing 1980-2004','housing 2004-',
           'office -1980','office 1980-2004','office 2004-',
           'laboratory -1980','laboratory 1980-2004','laboratory 2004-']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3, 3, 2, 3, 2],
      theta=categories,
      fill='toself',
      fillcolor='rgba(0, 100, 255, 0.6)',
      name='EUI'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2, 3, 2, 3, 2],
      theta=categories,
      fill='toself',
      fillcolor='rgba(255, 0, 0, 0.7)',
      name='COST'
))
fig.add_trace(go.Scatterpolar(
      r=[2, 2, 1, 1, 2, 4, 1, 1, 3],
      theta=categories,
      fill='toself',
      fillcolor='rgba(48, 180, 95, 0.5)',
      name='HEALTH'
))
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

# Set title
fig.update_layout(
    title_text="Time series with range slider and selectors"
)

fig.show()

# Calculate and Update Dataframe

In [ ]:
import pandas as pd
# Read csv file
df = pd.read_csv('kendall_new2.csv')

In [ ]:
# Calculate the differences of 3 index
df['dif1_EUI'] = df['tEUI_base'] - df['tEUI_con']
df['dif2_EUI'] = df['tEUI_base'] - df['tEUI_DE']
df['dif3_EUI'] = df['tEUI_base'] - df['tEUI_DEP']
df['dif1_CO2e'] = df['tCO2e_base'] - df['tCO2e_con']
df['dif2_CO2e'] = df['tCO2e_base'] - df['tCO2e_DE']
df['dif3_CO2e'] = df['tCO2e_base'] - df['tCO2e_DEP']
df['dif1_COST'] = df['tCOST_con'] 
df['dif2_COST'] = df['tCOST_DE'] 
df['dif3_COST'] = df['tCOST_DEP'] 

# Calculate the 3 total maximized sum
max_dif_EUI = df['dif3_EUI'].sum()
max_dif_CO2e = df['dif3_CO2e'].sum()
max_dif_COST = df['dif3_COST'].sum()

# Sort the data ascending by vintage
df_office = df.loc[df['type']=='Office'].sort_values(['vintage'], ascending=False)
df_housing = df.loc[df['type']=='Housing'].sort_values(['vintage'], ascending=False)
df_lab = df.loc[df['type']=='Laboratory'].sort_values(['vintage'], ascending=False)

# Max sums for each type and for each index
max_dif_EUI_housing = df_housing['dif3_EUI'].sum()
max_dif_EUI_office = df_office['dif3_EUI'].sum()
max_dif_EUI_lab = df_lab['dif3_EUI'].sum()
max_dif_CO2e_housing = df_housing['dif3_CO2e'].sum()
max_dif_CO2e_office = df_office['dif3_CO2e'].sum()
max_dif_CO2e_lab = df_lab['dif3_CO2e'].sum()
max_dif_COST_housing = df_housing['dif3_COST'].sum()
max_dif_COST_office = df_office['dif3_COST'].sum()
max_dif_COST_lab = df_lab['dif3_COST'].sum()

In [ ]:
# Define functions for converting differences into percentages in the max
def calculate_percent_office(level,index,val):
  if level == 'Conventional':
    if index == 'EUI':
      return df_office['dif1_EUI'].sample(frac=val).sum()/max_dif_EUI_office
    elif index == 'CO2e':
      return df_office['dif1_CO2e'].sample(frac=val).sum()/max_dif_CO2e_office
    else:
      return df_office['dif1_COST'].sample(frac=val).sum()/max_dif_COST_office
  elif level == 'Deep Energy':
    if index == 'EUI':
      return df_office['dif2_EUI'].sample(frac=val).sum()/max_dif_EUI_office
    elif index == 'CO2e':
      return df_office['dif2_CO2e'].sample(frac=val).sum()/max_dif_CO2e_office
    else:
      return df_office['dif2_COST'].sample(frac=val).sum()/max_dif_COST_office    
  elif level == 'Deep Energy Plus':
    if index == 'EUI':
      return df_office['dif3_EUI'].sample(frac=val).sum()/max_dif_EUI_office
    elif index == 'CO2e':
      return df_office['dif3_CO2e'].sample(frac=val).sum()/max_dif_CO2e_office
    else:
      return df_office['dif3_COST'].sample(frac=val).sum()/max_dif_COST_office 

def calculate_percent_housing(level,index,val):
  if level == 'Conventional':
    if index == 'EUI':
      return df_housing['dif1_EUI'].sample(frac=val).sum()/max_dif_EUI_housing
    elif index == 'CO2e':
      return df_housing['dif1_CO2e'].sample(frac=val).sum()/max_dif_CO2e_housing
    else:
      return df_housing['dif1_COST'].sample(frac=val).sum()/max_dif_COST_housing
  elif level == 'Deep Energy':
    if index == 'EUI':
      return df_housing['dif2_EUI'].sample(frac=val).sum()/max_dif_EUI_housing
    elif index == 'CO2e':
      return df_housing['dif2_CO2e'].sample(frac=val).sum()/max_dif_CO2e_housing
    else:
      return df_housing['dif2_COST'].sample(frac=val).sum()/max_dif_COST_housing    
  elif level == 'Deep Energy Plus':
    if index == 'EUI':
      return df_housing['dif3_EUI'].sample(frac=val).sum()/max_dif_EUI_housing
    elif index == 'CO2e':
      return df_housing['dif3_CO2e'].sample(frac=val).sum()/max_dif_CO2e_housing
    else:
      return df_housing['dif3_COST'].sample(frac=val).sum()/max_dif_COST_housing 

def calculate_percent_lab(level,index,val):
  if level == 'Conventional':
    if index == 'EUI':
      return df_lab['dif1_EUI'].sample(frac=val).sum()/max_dif_EUI_lab
    elif index == 'CO2e':
      return df_lab['dif1_CO2e'].sample(frac=val).sum()/max_dif_CO2e_lab
    else:
      return df_lab['dif1_COST'].sample(frac=val).sum()/max_dif_COST_lab
  elif level == 'Deep Energy':
    if index == 'EUI':
      return df_lab['dif2_EUI'].sample(frac=val).sum()/max_dif_EUI_lab
    elif index == 'CO2e':
      return df_lab['dif2_CO2e'].sample(frac=val).sum()/max_dif_CO2e_lab
    else:
      return df_lab['dif2_COST'].sample(frac=val).sum()/max_dif_COST_lab   
  elif level == 'Deep Energy Plus':
    if index == 'EUI':
      return df_lab['dif3_EUI'].sample(frac=val).sum()/max_dif_EUI_lab
    elif index == 'CO2e':
      return df_lab['dif3_CO2e'].sample(frac=val).sum()/max_dif_CO2e_lab
    else:
      return df_lab['dif3_COST'].sample(frac=val).sum()/max_dif_COST_lab
  elif level == 'Base':
      return value=='0.00'

# Interactive Radar Chart

In [ ]:
!pip install -q streamlit
!pip install pydeck

In [ ]:
"{:.2%}".format(max_dif_EUI)

Write the cell pytohn code into an app.py file

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import random
import plotly.express as px
import plotly.graph_objects as go

# Read csv file
df = pd.read_csv('kendall_new2.csv')

# Calculate the differences of 3 index
df['dif1_EUI'] = df['tEUI_base'] - df['tEUI_con']
df['dif2_EUI'] = df['tEUI_base'] - df['tEUI_DE']
df['dif3_EUI'] = df['tEUI_base'] - df['tEUI_DEP']
df['dif1_CO2e'] = df['tCO2e_base'] - df['tCO2e_con']
df['dif2_CO2e'] = df['tCO2e_base'] - df['tCO2e_DE']
df['dif3_CO2e'] = df['tCO2e_base'] - df['tCO2e_DEP']
df['dif1_COST'] = df['tCOST_con'] 
df['dif2_COST'] = df['tCOST_DE'] 
df['dif3_COST'] = df['tCOST_DEP'] 

# Calculate the 3 total maximized sum
max_dif_EUI = df['dif3_EUI'].sum()
max_dif_CO2e = df['dif3_CO2e'].sum()
max_dif_COST = df['dif3_COST'].sum()

# Sort the data ascending by vintage
df_office = df.loc[df['type']=='Office'].sort_values(['vintage'], ascending=False)
df_housing = df.loc[df['type']=='Housing'].sort_values(['vintage'], ascending=False)
df_lab = df.loc[df['type']=='Laboratory'].sort_values(['vintage'], ascending=False)

# Max sums for each type and for each index
max_dif_EUI_housing = df_housing['dif3_EUI'].sum()
max_dif_EUI_office = df_office['dif3_EUI'].sum()
max_dif_EUI_lab = df_lab['dif3_EUI'].sum()
max_dif_CO2e_housing = df_housing['dif3_CO2e'].sum()
max_dif_CO2e_office = df_office['dif3_CO2e'].sum()
max_dif_CO2e_lab = df_lab['dif3_CO2e'].sum()
max_dif_COST_housing = df_housing['dif3_COST'].sum()
max_dif_COST_office = df_office['dif3_COST'].sum()
max_dif_COST_lab = df_lab['dif3_COST'].sum()


# Define functions for converting differences into percentages in the max
def calculate_percent_office(level,index,val):
  if level == 'Conventional':
    if index == 'EUI':
      return df_office['dif1_EUI'].sample(frac=val).sum()/max_dif_EUI_office
    elif index == 'CO2e':
      return df_office['dif1_CO2e'].sample(frac=val).sum()/max_dif_CO2e_office
    else:
      return df_office['dif1_COST'].sample(frac=val).sum()/max_dif_COST_office
  elif level == 'Deep Energy':
    if index == 'EUI':
      return df_office['dif2_EUI'].sample(frac=val).sum()/max_dif_EUI_office
    elif index == 'CO2e':
      return df_office['dif2_CO2e'].sample(frac=val).sum()/max_dif_CO2e_office
    else:
      return df_office['dif2_COST'].sample(frac=val).sum()/max_dif_COST_office    
  elif level == 'Deep Energy Plus':
    if index == 'EUI':
      return df_office['dif3_EUI'].sample(frac=val).sum()/max_dif_EUI_office
    elif index == 'CO2e':
      return df_office['dif3_CO2e'].sample(frac=val).sum()/max_dif_CO2e_office
    else:
      return df_office['dif3_COST'].sample(frac=val).sum()/max_dif_COST_office 

def calculate_percent_housing(level,index,val):
  if level == 'Conventional':
    if index == 'EUI':
      return df_housing['dif1_EUI'].sample(frac=val).sum()/max_dif_EUI_housing
    elif index == 'CO2e':
      return df_housing['dif1_CO2e'].sample(frac=val).sum()/max_dif_CO2e_housing
    else:
      return df_housing['dif1_COST'].sample(frac=val).sum()/max_dif_COST_housing
  elif level == 'Deep Energy':
    if index == 'EUI':
      return df_housing['dif2_EUI'].sample(frac=val).sum()/max_dif_EUI_housing
    elif index == 'CO2e':
      return df_housing['dif2_CO2e'].sample(frac=val).sum()/max_dif_CO2e_housing
    else:
      return df_housing['dif2_COST'].sample(frac=val).sum()/max_dif_COST_housing    
  elif level == 'Deep Energy Plus':
    if index == 'EUI':
      return df_housing['dif3_EUI'].sample(frac=val).sum()/max_dif_EUI_housing
    elif index == 'CO2e':
      return df_housing['dif3_CO2e'].sample(frac=val).sum()/max_dif_CO2e_housing
    else:
      return df_housing['dif3_COST'].sample(frac=val).sum()/max_dif_COST_housing 

def calculate_percent_lab(level,index,val):
  if level == 'Conventional':
    if index == 'EUI':
      return df_lab['dif1_EUI'].sample(frac=val).sum()/max_dif_EUI_lab
    elif index == 'CO2e':
      return df_lab['dif1_CO2e'].sample(frac=val).sum()/max_dif_CO2e_lab
    else:
      return df_lab['dif1_COST'].sample(frac=val).sum()/max_dif_COST_lab
  elif level == 'Deep Energy':
    if index == 'EUI':
      return df_lab['dif2_EUI'].sample(frac=val).sum()/max_dif_EUI_lab
    elif index == 'CO2e':
      return df_lab['dif2_CO2e'].sample(frac=val).sum()/max_dif_CO2e_lab
    else:
      return df_lab['dif2_COST'].sample(frac=val).sum()/max_dif_COST_lab   
  elif level == 'Deep Energy Plus':
    if index == 'EUI':
      return df_lab['dif3_EUI'].sample(frac=val).sum()/max_dif_EUI_lab
    elif index == 'CO2e':
      return df_lab['dif3_CO2e'].sample(frac=val).sum()/max_dif_CO2e_lab
    else:
      return df_lab['dif3_COST'].sample(frac=val).sum()/max_dif_COST_lab
  elif level == 'Base':
      return value=='0.00'


st.write('# Retrofitting Kendall Square')

# Multiple trace radar chart

categories = ['housing','office','laboratory','housing']
retrofit = ['Base Case', 'Conventional', 'Deep Energy', 'Deep Energy Plus']

# Tick box
st.sidebar.markdown('#### Retrofitting Level')

eui = 'EUI'
cost = 'COST'
co2e = 'CO2e'

# Radar chart
def radar_chart(retrofit,val): 
    fig = go.Figure()

    # Fix the same numbers for line start and end
    r_housing_eui = round(calculate_percent_housing(retrofit,eui,val),2)
    r_housing_cost = round(calculate_percent_housing(retrofit,cost,val),2)
    r_housing_co2e = round(calculate_percent_housing(retrofit,co2e,val),2)

    fig.add_trace(go.Scatterpolar(
          r=[r_housing_eui,
       round(calculate_percent_office(retrofit,eui,val),2),
       round(calculate_percent_lab(retrofit,eui,val),2),
       r_housing_eui],
          theta=categories,
          type= 'scatterpolar',
          mode='lines',
          line_color='rgba(92, 140, 255, 0.8)',
          #fill='toself',
          #fillcolor='rgba(92, 140, 255, 0.8)',
          name='EUI'
    ))
    fig.add_trace(go.Scatterpolar(
          r=[r_housing_cost,
       round(calculate_percent_office(retrofit,cost,val),2),
       round(calculate_percent_lab(retrofit,cost,val),2),
       r_housing_cost],
          theta=categories,
          type= 'scatterpolar',
          mode='lines',
          line_color='rgba(255, 56, 75, 0.66)',
          #fill='toself',
          #fillcolor='rgba(255, 212, 97, 0.6)',
          name='COST'
    ))
    fig.add_trace(go.Scatterpolar(
          r=[r_housing_co2e,
       round(calculate_percent_office(retrofit,co2e,val),2),
       round(calculate_percent_lab(retrofit,co2e,val),2),
       r_housing_co2e],
          theta=categories,
          type= 'scatterpolar',
          mode='lines',
          line_color='rgba(0, 130, 120, 0.5)',
          #fill='toself',
          #fillcolor='rgba(112, 230, 195, 0.6)',
          name='HEALTH'
    ))
    fig.update_layout(
     polar=dict(
        radialaxis=dict(
          visible=True,
          range=[0,1]
        )),
      showlegend=False
    )
    st.write(fig)


if __name__ == '__main__':
    val = st.sidebar.slider('Select value',0.0,1.0,0.1)
    RL = st.sidebar.radio('Select the retrofit level', retrofit, index=1)
    radar_chart(RL,val)

Create local tunnel

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501